In [61]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install xmltodict

In [ ]:
!ls

 1.jpg				   impy-0.1-py3-none-any.whl.2
 augmentation_configuration.json  'kaggle (1).json'
 cars_dataset			   kaggle.json
'Colab Notebooks'		   pascal_voc
 corped_images.ipynb		   PASCAL_VOC
 data				   pascal-voc-2007.zip
 data1				   voc2017
 folder				   voctest_06-nov-2007
 images				   voctrainval_06-nov-2007
 impy-0.1-py3-none-any.whl	   VOCtrainval_06-Nov-2007
 impy-0.1-py3-none-any.whl.1	   yolov5


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"deepzsenu","key":"5a93c503d3d0802f7c6862cc201899a4"}'}

In [ ]:
!mkdir ~/.kaggle 

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d zaraks/pascal-voc-2007

 99% 1.64G/1.65G [00:17<00:00, 108MB/s] 
100% 1.65G/1.65G [00:17<00:00, 102MB/s]


In [ ]:
!unzip pascal-voc-2007.zip

Streaming output truncated to the last 5000 lines.
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001729.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001730.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001732.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001733.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001734.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001738.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001739.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001741.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001746.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001747.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001749.jpg  
  inflating: voctrainval_06-nov-2007/VOCdevkit/VOC2007/JPEGImages/001750.jpg  
 

In [ ]:
import glob
import os
import xmltodict
import json
import pprint

from PIL import Image
import numpy as np
import sys

import csv

In [ ]:
# default format can be changed as needed
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList


In [ ]:
# Printing results
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# Look for XML files and parses then as if they were Pascal VOC Files
def process():
    # Finds all XML files on data/ and append to list
    pascal_voc_contents = []
    os.chdir("data")

    print("Found {} files in data directory!".format(
        str(len(glob.glob("*.xml")))))
    for file in glob.glob("*.xml"):
        f_handle = open(file, 'r')
        print("Parsing file '{}'...".format(file))
        pascal_voc_contents.append(xmltodict.parse(f_handle.read()))

    # Process each file individually
    for index in pascal_voc_contents:
        image_file = index['annotation']['filename']
        # If there's a corresponding file in the folder,
        # process the images and save to output folder
        if os.path.isfile(image_file):
            extractDataset(index['annotation'])
        else:
            print("Image file '{}' not found, skipping file...".format(image_file))


In [164]:
# Extract image samples and save to output dir
def extractDataset(dataset):
    print("Found {} objects on image '{}'...".format(
        len(dataset['object']), dataset['filename']))

    # Open image and get ready to process
    img = Image.open(dataset['filename'])

    # Create output directory
    save_dir = "objects"
    try:
        os.mkdir(save_dir)
    except:
        pass
    # Image name preamble
    sample_preamble = save_dir + "/" + dataset['filename'].split('.')[0] + "_"
    # Image counter
    i = 0

    # Run through each item and save cut image to output folder
    for item in dataset['object']:
        # Convert str to integers
        bndbox = dict([(a, int(b)) for (a, b) in item['bndbox'].items()])
        # Crop image
        img_file = img.crop((bndbox['xmin'], bndbox['ymin'],
                       bndbox['xmax'], bndbox['ymax']))
        # Save
        img_file.save(sample_preamble + str(i) + '.jpg')

        #saving as csv form
        width, height = img_file.size
        format = img_file.format
        mode = img_file.mode

        img_grey = img_file.convert('L')
        value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
        value = value.flatten()
        print(value)
        with open("/content/drive/MyDrive/data/csv/images.csv", 'a') as f:
          writer = csv.writer(f, value, delimiter = " ")
          writer.writerow(value )
        i = i + 1


In [165]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [166]:
if __name__ == '__main__':
  process()

Found 4 files in data directory!
Parsing file '000001.xml'...
Parsing file '000003.xml'...
Parsing file '000004.xml'...
Parsing file '000006.xml'...
Found 2 objects on image '000001.jpg'...
[10 21 11 ... 57 57 56]
[ 2  1  1 ... 61 52 52]
Found 2 objects on image '000003.jpg'...
[156 219 192 ... 137 124 104]
[ 48  59 121 ...  88  88  90]
Found 7 objects on image '000004.jpg'...
[ 46  33  43 ... 102 104 110]
[ 64  60  61 ...  16  89 255]
[35 29 59 ...  8 18 11]
[ 79 112  49 ...  16  11  24]
[57 98 69 ... 11  7  9]
[50 62 36 ... 33  9 12]
[ 44  99  36  37  67  54  54  62  69  42  48  66  48  67  76  81  42  54
  51  26 216  50  41  65  55  59  40  13  12  18  20  98 111  73  58 116
 148  53 115  36  44  47  69  67  62  62  45  43  60  40  30  81  52  39
  38  40  47 136  40  48  58  44  65  36  21  12   4  13 122 154  80  58
 120 136  47  75  32  41  36  37  35  63  64  42  62  64  58  49  94  65
  48  44  71  28  69  59  54  77  50  62  36  26   1  39   8  90 180  92
  48 124 133 124 224

In [167]:
import pandas as pd
import numpy as np

In [168]:
header_list = ["pixels"]
df = pd.read_csv("/content/drive/MyDrive/data/csv/images.csv" ,names = header_list )
#print(df.shape)
#df.head()

In [169]:
print(df.shape)

(19, 1)


In [171]:
df.head()

,pixels
0,10 21 11 15 15 5 14 16 20 3 3 5 1 10 2 9 11 7 ...
1,2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
2,156 219 192 186 191 193 199 208 196 212 211 21...
3,48 59 121 113 77 84 95 98 89 87 94 84 88 98 10...
4,46 33 43 52 100 141 200 130 166 190 141 201 65...


In [176]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [177]:
pascal_voc_contents = []
os.chdir("data")
print("Found {} files in data directory!".format(
str(len(glob.glob("*.xml")))))
for file in glob.glob("*.xml"):
    f_handle = open(file, 'r')
    print("Parsing file '{}'...".format(file))
    pascal_voc_contents.append(xmltodict.parse(f_handle.read()))


Found 4 files in data directory!
Parsing file '000001.xml'...
Parsing file '000003.xml'...
Parsing file '000004.xml'...
Parsing file '000006.xml'...


In [183]:
pascal_voc_contents

[OrderedDict([('annotation',
               OrderedDict([('folder', 'VOC2007'),
                            ('filename', '000001.jpg'),
                            ('source',
                             OrderedDict([('database', 'The VOC2007 Database'),
                                          ('annotation', 'PASCAL VOC2007'),
                                          ('image', 'flickr'),
                                          ('flickrid', '341012865')])),
                            ('owner',
                             OrderedDict([('flickrid', 'Fried Camels'),
                                          ('name', 'Jinky the Fruit Bat')])),
                            ('size',
                             OrderedDict([('width', '353'),
                                          ('height', '500'),
                                          ('depth', '3')])),
                            ('segmented', '0'),
                            ('object',
                             [OrderedDic

In [189]:
for index in pascal_voc_contents:
        name = index['annotation']['object']
        print(name)

[OrderedDict([('name', 'dog'), ('pose', 'Left'), ('truncated', '1'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '48'), ('ymin', '240'), ('xmax', '195'), ('ymax', '371')]))]), OrderedDict([('name', 'person'), ('pose', 'Left'), ('truncated', '1'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '8'), ('ymin', '12'), ('xmax', '352'), ('ymax', '498')]))])]
[OrderedDict([('name', 'sofa'), ('pose', 'Unspecified'), ('truncated', '0'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '123'), ('ymin', '155'), ('xmax', '215'), ('ymax', '195')]))]), OrderedDict([('name', 'chair'), ('pose', 'Left'), ('truncated', '0'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '239'), ('ymin', '156'), ('xmax', '307'), ('ymax', '205')]))])]
[OrderedDict([('name', 'car'), ('pose', 'Frontal'), ('truncated', '0'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '13'), ('ymin', '311'), ('xmax', '84'), ('ymax', '362')]))]), OrderedDict([('name', 'car'), ('pose', 'Unspecified'), ('trunc

In [106]:
from xml.etree.ElementTree import parse

In [117]:
import os
list = os.listdir('/content/drive/MyDrive/data/')

In [119]:
print(list)

['000001.xml', '000003.xml', '000004.xml', '000006.xml', '000001.jpg', '000003.jpg', '000004.jpg', '000006.jpg', 'objects', 'csv', '.ipynb_checkpoints']


In [193]:
import xml.etree.ElementTree as ET
names = []
for i in range(4):
  root = ET.parse("/content/drive/MyDrive/data/"+list[i]).getroot()
  for type_tag in root.findall('object'):
    name = type_tag.find('name').text
    names.append(name)

In [197]:
data = pd.DataFrame(names, columns = ["name"])
data.head()

,name
0,dog
1,person
2,sofa
3,chair
4,car


In [199]:
names = data["name"]
df = df.join(names)

df.head()

,pixels,name
0,10 21 11 15 15 5 14 16 20 3 3 5 1 10 2 9 11 7 ...,dog
1,2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,person
2,156 219 192 186 191 193 199 208 196 212 211 21...,sofa
3,48 59 121 113 77 84 95 98 89 87 94 84 88 98 10...,chair
4,46 33 43 52 100 141 200 130 166 190 141 201 65...,car


In [200]:
df = df[["name","pixels"]]

In [201]:
df.head(1)

,name,pixels
0,dog,10 21 11 15 15 5 14 16 20 3 3 5 1 10 2 9 11 7 ...
